## Data Understanding

### Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
#read the raw data
raw_df = pd.read_pickle('../data/telecom_xdr.pkl')
#drop any duplicate rows
raw_df_droped_dup = raw_df.drop_duplicates()
#associeted column descriptions
colum_descript = pd.read_excel('../resources/Field Descriptions.xlsx')

In [3]:
raw_df.head(100)

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.311448e+19,4/22/2019 7:38,804.0,4/25/2019 0:01,199.0,231780.0,2.082017e+14,3.361513e+10,3.520031e+13,L73684B,...,7438887.0,13154981.0,753238.0,9887128.0,124180544.0,14967821.0,350039548.0,14932623.0,53052136.0,139043208.0
96,1.311448e+19,4/22/2019 7:54,676.0,4/25/2019 0:01,176.0,230839.0,2.082014e+14,3.366495e+10,8.636750e+13,D73608B,...,3334865.0,14454366.0,17917894.0,1530332.0,293492651.0,1362536.0,495376623.0,11684759.0,30713366.0,329159454.0
97,7.349883e+18,4/22/2019 8:00,810.0,4/25/2019 2:22,294.0,238915.0,2.082010e+14,3.366403e+10,3.586111e+13,D73605A,...,23039020.0,21457678.0,1114842.0,13958301.0,516508263.0,8432981.0,624397048.0,9207067.0,57680454.0,546747900.0
98,1.311448e+19,4/22/2019 8:15,473.0,4/25/2019 2:55,731.0,239954.0,2.082010e+14,3.366875e+10,3.533251e+13,L76864A,...,14851113.0,1312259.0,11225027.0,15097330.0,522050751.0,1138947.0,281699270.0,3082007.0,21749419.0,549470626.0


In [4]:
# column names missmatch b/n data and description
[x for x in colum_descript['Fields'].tolist() if x not in raw_df.columns.tolist()]

['bearer id',
 'Dur. (s)',
 'YouTube DL (Bytes)',
 'YouTube UL (Bytes)',
 'Other DL',
 'Other UL']

In [5]:
[x for x in raw_df.columns.tolist() if x not in colum_descript['Fields'].tolist()]

['Bearer Id',
 'Youtube DL (Bytes)',
 'Youtube UL (Bytes)',
 'Other DL (Bytes)',
 'Other UL (Bytes)']

In [6]:
#Match columns in the description df to the data df
raw_df_renamed = raw_df.rename(columns={'Bearer Id':'bearer id' , 'Youtube DL (Bytes)':'YouTube DL (Bytes)', 'Youtube UL (Bytes)':'YouTube UL (Bytes)', 'Other DL (Bytes)':'Other DL', 'Other UL (Bytes)':'Other UL'})
del raw_df
gc.collect()

1303

### Data Pre-Processing
Data preprocessing is an integral step in Machine Learning as the quality of data and the useful information that can be derived from it directly affects the ability of our model to learn; therefore, it is extremely important that we preprocess our data before feeding it into our model.
#### Handling Null Values

In [7]:
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.prod(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The Diabetes dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

def columns_missing_most_values(df, percentage):
    colum_wise = df.isnull().sum() * 100 /df.shape[0]
    col_names = colum_wise[colum_wise > percentage].index.tolist() 
    print(" {} columns are missing moethan {}% of their values\nThey are:\n".format(len(col_names), percentage))
    with pd.option_context('expand_frame_repr', False):
        droped = colum_descript.loc[colum_descript['Fields'].isin(col_names)]
        droped['% Missing'] = [colum_wise.get(col_nm) for col_nm in col_names]
        print (droped)
    return col_names

percent_missing(raw_df_renamed)
# drop columns with more than 30% missing values
to_drop = columns_missing_most_values(raw_df_renamed, 30)
raw_df_droped = raw_df_renamed.drop(to_drop, axis=1)
raw_df_droped.shape
del raw_df_renamed
gc.collect()


The Diabetes dataset contains 12.5 % missing values.
 10 columns are missing moethan 30% of their values
They are:

                                      Fields                                        Description  % Missing
15               TCP DL Retrans. Vol (Bytes)  TCP volume of Downlink packets detected as ret...  58.763608
16               TCP UL Retrans. Vol (Bytes)  TCP volume of Uplink packets detected as retra...  64.432237
25                           HTTP DL (Bytes)  HTTP data volume (in Bytes) received by the MS...  54.315638
26                           HTTP UL (Bytes)  HTTP data volume (in Bytes) sent by the MS dur...  54.539636
32           Nb of sec with 125000B < Vol DL              Number of seconds with IP Volume DL >  65.024900
33     Nb of sec with 1250B < Vol UL < 6250B      Number of seconds with IP Volume UL between …  61.928920
34  Nb of sec with 31250B < Vol DL < 125000B      Number of seconds with IP Volume DL between …  62.390251
35            Nb of sec with

/tmp/ipykernel_6236/2798157602.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  droped['% Missing'] = [colum_wise.get(col_nm) for col_nm in col_names]


0

In [8]:
def columns_missing_values(df):
    colum_wise = df.isnull().sum() * 100 /df.shape[0]
    missing_col_names = colum_wise[colum_wise > 0].index.tolist()
    complete_col_names = [cn for cn in df.columns if cn not in missing_col_names]
    print(" {} columns are missing atleast 1 value, while {} have no missing values.\nThe columns missing values are:\n".format(len(missing_col_names), len(complete_col_names)))
    with pd.option_context('expand_frame_repr', False):
        with_missing = colum_descript.loc[colum_descript['Fields'].isin(missing_col_names)]
        with_missing['% Missing'] = [colum_wise.get(col_nm) for col_nm in missing_col_names]
        print (with_missing.sort_values(by=['% Missing']))
        print("\n The columns with full data are:\n")
        complete = colum_descript.loc[colum_descript['Fields'].isin(complete_col_names)]
        complete['% Missing'] = [colum_wise.get(col_nm) for col_nm in complete_col_names]
        print (complete)
    return missing_col_names, complete_col_names
# inspect the rest of the missimng values(their description)
to_fill, complet = columns_missing_values(raw_df_droped)

 31 columns are missing atleast 1 value, while 14 have no missing values.
The columns missing values are:

                            Fields                                        Description  % Missing
55                Total UL (Bytes)  Data volume (in Bytes) sent by the MS during t...   0.000667
1                        Dur. (ms)                  Total Duration of the xDR (in ms)   0.000667
2                            Start      Start time of the xDR (first frame timestamp)   0.000667
3                         Start ms  Milliseconds offset of start time for the xDR ...   0.000667
4                              End         End time of the xDR (last frame timestamp)   0.000667
5                           End ms  Milliseconds offset of end time of the xDR (la...   0.000667
27       Activity Duration DL (ms)  Activity Duration for downlink (ms) - excludin...   0.000667
54                Total DL (Bytes)  Data volume (in Bytes) received by the MS duri...   0.000667
14         Avg Beare

/tmp/ipykernel_6236/1058487574.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_missing['% Missing'] = [colum_wise.get(col_nm) for col_nm in missing_col_names]
/tmp/ipykernel_6236/1058487574.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete['% Missing'] = [colum_wise.get(col_nm) for col_nm in complete_col_names]


##### The 45 data columns can be grouped in to three different data-types:
* Categorical:

['Handset Type', 'Handset Manufacturer', 'Last Location Name', 'IMEI', 'MSISDN/Number', 'IMSI', 'bearer id']
* Time Series:

['Start ms', 'Start', 'End ms', 'End']
* Numerical contin:

['DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)']
* Numerical descrite:

['Dur. (ms)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)', 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)', 'Dur. (ms).1', 'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B',
 'Social Media DL (Bytes)', 'Social Media UL (Bytes)', 'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
 'Email UL (Bytes)', 'YouTube DL (Bytes)', 'YouTube UL (Bytes)', 'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
 'Gaming UL (Bytes)', 'Other DL', 'Other UL', 'Total UL (Bytes)', 'Total DL (Bytes)']

In [9]:
categorical_columns = ['Handset Type', 'Handset Manufacturer', 'Last Location Name', 'IMEI', 'MSISDN/Number', 'IMSI', 'bearer id']
time_series_columns = ['Start ms', 'Start', 'End ms', 'End']
numer_cont_colmns = ['DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 
                    'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)']
numer_dicrt_colmns = ['Dur. (ms)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 
                      'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 
                      'UL TP < 10 Kbps (%)','10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)', 
                      'Activity Duration DL (ms)','Activity Duration UL (ms)', 'Dur. (ms).1', 'Nb of sec with Vol DL < 6250B',
                      'Nb of sec with Vol UL < 1250B','Social Media DL (Bytes)', 'Social Media UL (Bytes)', 'Google DL (Bytes)', 
                      'Google UL (Bytes)', 'Email DL (Bytes)','Email UL (Bytes)', 'YouTube DL (Bytes)', 'YouTube UL (Bytes)', 'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
                      'Gaming UL (Bytes)', 'Other DL', 'Other UL', 'Total UL (Bytes)', 'Total DL (Bytes)']

In [10]:
#print sorted no_of_unique values per column 
def columns_uniqueness(df):
    col_wise_unique = df.nunique(dropna=True)
    col_names = col_wise_unique.index.tolist()
    uniques = colum_descript.loc[colum_descript['Fields'].isin(col_names)]
    uniques['uniques'] = [col_wise_unique.get(col_nm) for col_nm in col_names]
    uniques.sort_values(by=['uniques'])
    with pd.option_context('expand_frame_repr', False):
        print (uniques.sort_values(by=['uniques']))

def print_sorted_no_uniques(df):
    #for the columsn with the missing values
    print("No of unique values in the columns missing values")
    df_missing_values = df.drop(complet, axis=1)
    columns_uniqueness(df_missing_values)

    #for the columns without any missing values
    print("\nNo of unique values in the columns without any missing values")
    df_complete_values = df.drop(to_fill, axis=1)
    columns_uniqueness(df_complete_values)

print_sorted_no_uniques(raw_df_droped)

No of unique values in the columns missing values
                            Fields                                        Description  uniques
24            UL TP > 300 Kbps (%)  Duration ratio when Bearer Uplink Throughput > ….       63
23  50 Kbps < UL TP < 300 Kbps (%)  Duration ratio when Bearer Uplink Throughput r...       68
19   250 Kbps < DL TP < 1 Mbps (%)  Duration ratio when Bearer Downlink Throughput...       73
18  50 Kbps < DL TP < 250 Kbps (%)  Duration ratio when Bearer Downlink Throughput...       84
22   10 Kbps < UL TP < 50 Kbps (%)  Duration ratio when Bearer Uplink Throughput r...       85
20              DL TP > 1 Mbps (%)  Duration ratio when Bearer Downlink Throughput...       85
21             UL TP < 10 Kbps (%)  Duration ratio when Bearer Uplink Throughput < ….       98
17             DL TP < 50 Kbps (%)  Duration ratio when Bearer Downlink Throughput...      100
30            Handset Manufacturer                              Handset manufacturer       170


/tmp/ipykernel_6236/1718879088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniques['uniques'] = [col_wise_unique.get(col_nm) for col_nm in col_names]
/tmp/ipykernel_6236/1718879088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniques['uniques'] = [col_wise_unique.get(col_nm) for col_nm in col_names]


##### !! The is no single unique column to be used as key

In [11]:
# Fill all the 8 contineous numerical columns(% ratios) with 0.0 as it is a probaility dist.(needs to addup to one)
values_0 = { k:v for (k,v) in zip(numer_cont_colmns, [0.0] * 8)} 
raw_df_droped_fill_0 = raw_df_droped.fillna(value=values_0)

In [12]:
# Fill the 3 string categorical columns with 'undefined'
values_1 = { k:v for (k,v) in zip(['Handset Type', 'Handset Manufacturer', 'Last Location Name'], ['undefined'] * 3)} 
raw_df_droped_fill_1 = raw_df_droped_fill_0.fillna(value=values_1)

In [13]:
# Fill the rest of the categorical columns with 0
values_2 = { k:v for (k,v) in zip(['IMEI', 'MSISDN/Number', 'IMSI', 'bearer id'],[0]*3)}
raw_df_droped_fill_2 = raw_df_droped_fill_1.fillna(value=values_2)

In [15]:
# Fill the decrit numrical columns with 0, as they represent either a data-size in Bytes or time in Sec/mili Sec
values_3 = { k:v for (k,v) in zip(numer_dicrt_colmns,[0]*len(numer_dicrt_colmns))}
raw_df_droped_fill_3 = raw_df_droped_fill_2.fillna(value=values_3)

In [16]:
# Fill the Time series(what are left) columns formawrd
raw_df_droped_fill_4 = raw_df_droped_fill_3.fillna( method='ffill')
del raw_df_droped_fill_3, raw_df_droped_fill_2, raw_df_droped_fill_1, raw_df_droped_fill_0, values_0, values_1, values_2, values_3
gc.collect()

/tmp/ipykernel_6236/4011880599.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  raw_df_droped_fill_4 = raw_df_droped_fill_3.fillna( method='ffill')


0

In [17]:
# To check for missing values
to_fill, complet = columns_missing_values(raw_df_droped_fill_4)

 0 columns are missing atleast 1 value, while 45 have no missing values.
The columns missing values are:

Empty DataFrame
Columns: [Fields, Description, % Missing]
Index: []

 The columns with full data are:

                            Fields                                        Description  % Missing
0                        bearer id                             xDr session identifier        0.0
1                        Dur. (ms)                  Total Duration of the xDR (in ms)        0.0
2                            Start      Start time of the xDR (first frame timestamp)        0.0
3                         Start ms  Milliseconds offset of start time for the xDR ...        0.0
4                              End         End time of the xDR (last frame timestamp)        0.0
5                           End ms  Milliseconds offset of end time of the xDR (la...        0.0
7                             IMSI           International Mobile Subscriber Identity        0.0
8              

/tmp/ipykernel_6236/1058487574.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete['% Missing'] = [colum_wise.get(col_nm) for col_nm in complete_col_names]


##### Standardize the numerical columns with Z-score to get mean of 0 and sd of 1

In [244]:
df_standard = raw_df_droped_fill_4
for col in numer_dicrt_colmns + numer_cont_colmns:
    df_standard[col] = (raw_df_droped_fill_4[col] - raw_df_droped_fill_4[col].mean())/raw_df_droped_fill_4[col].std(ddof=0)
del raw_df_droped_fill_4
gc.collect()

##### One-Hot-Encode the categoricals

In [1]:
df_ohe = pd.get_dummies(df_standard, columns = categorical_columns[:3])

NameError: name 'pd' is not defined

## Modelling


In [ ]:
y= df_clean['readmitted']
len(y)

In [ ]:
x = df_clean.drop(['readmitted'],axis=1)

In [ ]:
len(x)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt



# Feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(x, y)

# Summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)

for i in range(len(fit.scores_)):
	print('Feature %d: %f' % (i, fit.scores_[i]))
# plot the scores
plt.bar([i for i in range(len(fit.scores_))], fit.scores_)
plt.show()
#features = fit.transform(x)
# Summarize selected features
#print(features[0:5,:])

In [ ]:
selected_features=x[['discharge_disposition_id', 'time_in_hospital', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'change', 'diabetesMed']]
len(selected_features)

In [ ]:
selected_features.head()

In [ ]:
len(y)

In [ ]:
y.to_csv('label.csv')

In [ ]:
selected_features.to_csv('features.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from joblib import dump,load
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import pickle

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(selected_features,y,test_size=0.2,random_state=40)

# creating scaler scale var.
norm = MinMaxScaler()
# fit the scal
norm_fit = norm.fit(xtrain)
pickle.dump(norm_fit, open("scaler.pkl", 'wb'))
dump(norm_fit,'scaler.joblib')
# transfromation of trainig data
scal_xtrain = norm_fit.transform(xtrain)

# transformation of testing data
scal_xtest = norm_fit.transform(xtest)
print(scal_xtrain)

In [ ]:
  
# create model variable
rnd = RandomForestClassifier()
  
# fit the model
fit_rnd = rnd.fit(xtrain,ytrain)  

In [ ]:
#predicting our target variable
y_predict = fit_rnd.predict(xtest)

#predicting the probability of our target variable
y_predict_probabilty = fit_rnd.predict_proba(xtest)

In [ ]:
print(list(y_predict))

In [ ]:
print(y_predict_probabilty)

In [ ]:
# checking the accuracy score
rnd_score = fit_rnd.score(xtest,ytest) 
  
print('score of model is : ',rnd_score)
  
pickle.dump(fit_rnd, open("model.pkl", 'wb'))
#dump(rnd, 'model.joblib')

In [ ]:
from sklearn.metrics import classification_report
x_predict = list(rnd.predict(xtest))
print(classification_report(ytest, x_predict))
#df = {'predicted':x_predict,'orignal':ytest}
#df